In [1]:
from PIL import Image
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Activation, Flatten

Using TensorFlow backend.


In [2]:
img_dir_list = ['PIE/1/*.jpg', 'PIE/4/*.jpg', 'PIE/5/*.jpg', \
               'PIE/19/*.jpg', 'PIE/21/*.jpg', 'PIE/23/*.jpg', \
               'PIE/25/*.jpg', 'PIE/29/*.jpg', 'PIE/33/*.jpg', \
               'PIE/39/*.jpg', 'PIE/40/*.jpg', 'PIE/44/*.jpg', \
               'PIE/45/*.jpg', 'PIE/46/*.jpg', 'PIE/48/*.jpg', \
               'PIE/52/*.jpg', 'PIE/57/*.jpg', 'PIE/58/*.jpg', \
               'PIE/59/*.jpg', 'PIE/67/*.jpg']
image_list = []
image_own = []
train_list = []
train_own = []
test_list = []
test_own = []
image_label = []
own_label = [20] * 10

for i in range (0, 20):
    for filename in glob.glob(img_dir_list[i]):
        im = Image.open(filename)
        arr = np.array(im).flatten()
        image_list.append(arr)
        image_label.append(i)
for filename in glob.glob('own_image/*.jpg'):
    im = Image.open(filename)
    arr = np.array(im).flatten()
    image_own.append(arr)
    
image_list = np.asarray(image_list)
image_own = np.asarray(image_own)
image_label = np.asarray(image_label)
own_label = np.asarray(own_label)

train_list, test_list, train_label, test_label = train_test_split(image_list, image_label, test_size = 0.3)
train_list = np.asarray(train_list)
test_list = np.asarray(test_list)
train_label = np.asarray(train_label)
test_label = np.asarray(test_label)

train_own, test_own, train_own_label, test_own_label = train_test_split(image_own, own_label, test_size = 0.3)
train_own = np.asarray(train_own)
test_own = np.asarray(test_own)
train_own_label = np.asarray(train_own_label)
test_own_label = np.asarray(test_own_label)

random_indices = np.arange(0, train_list.shape[0])
np.random.shuffle(random_indices)
train_sample = train_list[random_indices[:500]]
train_sample_label = train_label[random_indices[:500]]
# print(train_sample)
train_sample = np.concatenate((train_sample, train_own))
train_sample_label = np.concatenate((train_sample_label, train_own_label))
test_list = np.concatenate((test_list, test_own))
test_label = np.concatenate((test_label, test_own_label))

train_sample = np.asarray(train_sample)
test_list = np.asarray(test_list)

sc = preprocessing.StandardScaler()
train_sample_prep = sc.fit_transform(train_sample)
test_list_prep = sc.transform(test_list)
train_list_prep = sc.fit_transform(train_list)

train_sample = train_sample_prep.reshape(train_sample_prep.shape[0], 32, 32, 1)
test_list = test_list_prep.reshape(test_list_prep.shape[0], 32, 32, 1)
train_list = train_list_prep.reshape(train_list_prep.shape[0], 32, 32, 1)

In [3]:
print(train_sample[0].shape)

(32, 32, 1)


In [4]:
test_label = to_categorical(test_label, num_classes = 21)
train_sample_label = to_categorical(train_sample_label, num_classes = 21)
train_label = to_categorical(train_label, num_classes = 21)

In [5]:
model = Sequential()

model.add(Conv2D(filters=20, kernel_size=5, strides=1, padding="same", input_shape=(32, 32, 1)))
model.add(MaxPool2D(pool_size=2, strides=2))

model.add(Conv2D(filters=50, kernel_size=5, strides=1, padding='same'))
model.add(MaxPool2D(pool_size=2, strides=2))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))

model.add(Dense(21))
model.add(Activation('softmax'))
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 20)        520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 50)        25050     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1600500   
_________________________________________________________________
activation_1 (Activation)    (None, 500)              

In [6]:
model.fit(train_sample, train_sample_label, epochs=20, batch_size=32, validation_data=(test_list, test_label),
              shuffle=True, verbose=2, )

Train on 507 samples, validate on 1023 samples
Epoch 1/20
 - 1s - loss: 2.9529 - accuracy: 0.1262 - val_loss: 2.8453 - val_accuracy: 0.1779
Epoch 2/20
 - 1s - loss: 2.6057 - accuracy: 0.2899 - val_loss: 2.6618 - val_accuracy: 0.2434
Epoch 3/20
 - 1s - loss: 2.2918 - accuracy: 0.4280 - val_loss: 2.3990 - val_accuracy: 0.3773
Epoch 4/20
 - 1s - loss: 1.9403 - accuracy: 0.6036 - val_loss: 2.0568 - val_accuracy: 0.5357
Epoch 5/20
 - 1s - loss: 1.5354 - accuracy: 0.7337 - val_loss: 1.6927 - val_accuracy: 0.6246
Epoch 6/20
 - 1s - loss: 1.1556 - accuracy: 0.8185 - val_loss: 1.3558 - val_accuracy: 0.6970
Epoch 7/20
 - 1s - loss: 0.8656 - accuracy: 0.8698 - val_loss: 1.0705 - val_accuracy: 0.7625
Epoch 8/20
 - 1s - loss: 0.6451 - accuracy: 0.8994 - val_loss: 0.9018 - val_accuracy: 0.7801
Epoch 9/20
 - 1s - loss: 0.4801 - accuracy: 0.9231 - val_loss: 0.7473 - val_accuracy: 0.8270
Epoch 10/20
 - 1s - loss: 0.3831 - accuracy: 0.9487 - val_loss: 0.6500 - val_accuracy: 0.8514
Epoch 11/20
 - 1s - lo

In [7]:
loss, accuracy = model.evaluate(train_sample, train_sample_label, verbose = 0)
print(accuracy * 100, loss)
loss,  accuracy = model.evaluate(test_list, test_label, verbose = 0)
print(accuracy* 100, loss)
loss,  accuracy = model.evaluate(train_list, train_label, verbose = 0)
print(accuracy* 100, loss)

100.0 0.0389726777327719
89.83381986618042 0.3420039644047895
91.26050472259521 0.3137206617774082
